## Tuning of the evaluator

The purpose of this notebook is to evaluate how close are evaluations of LLM generated solutions between Human and LLM.

* It allows to create a tuning set by selecting random tasks from the whole benchmark set.
* It contains code for comparing the match scores given to the LLM generated solutions by human and LLM by using confusion matrix

If the difference is too high, the adjustments to the prompt in the evaluation.py file might be needed. The notebook contains code to facilitate analysis of cases where the match scores differed. 

When ready, the evaluator code is used in the "Benchmarking" notebook.

##### Imports

In [13]:
# !pip install -r requirements.txt

In [1]:
import os, sys
import json
import zipfile
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from io import StringIO
import importlib
from collections import defaultdict
import random
from pprint import pprint, pformat, PrettyPrinter
import re
from tqdm import tqdm

In [ ]:
from geobenchx.constants import DATA_FOLDER, RESULTS_FOLDER, MODEL_CLAUDE, MODEL_GEMINI_ADV, MODEL_GPT_41, MODEL_GPT_mini, MODEL_O3, MODEL_CLAUDE_ADV4
import geobenchx.dataclasses
importlib.reload(geobenchx.dataclasses)
from geobenchx.dataclasses import TaskSet, Task, Solution

import geobenchx.utils
importlib.reload(geobenchx.utils)
from geobenchx.utils import generate_timestamp_id, get_dataframe_info, get_solution_code


import geobenchx.agent
importlib.reload(geobenchx.agent)
from geobenchx.agent import execute_task

import geobenchx.evaluation
importlib.reload(geobenchx.evaluation)
from geobenchx.evaluation import score_task_solution, generate_eval_stats_evaluator, score_solutions_set

#### Selecting random tasks for annotation

In [4]:
source_tasks = 'tasks_and_reference_solutions.json' # name of file with tasks and reference solutions (ground truth solutions)

In [3]:
tuning_tasks_filename = 'evaluator_tuning_set.json' # name for file with tasks with references solution, candidate solution and manual match score

In [4]:
# Selecting the tasks from 'source_tasks' for the evaluator tuning set or reading them from already exsisting file with tuning tasks set

if os.path.exists(os.path.join(DATA_FOLDER, tuning_tasks_filename)): 
    evaluator_tuning_tasks = TaskSet.read_from_file(tuning_tasks_filename, DATA_FOLDER)   
else:
    tasks = TaskSet.read_from_file(source_tasks, DATA_FOLDER)
    evaluator_tuning_tasks = tasks.sample_stratified(40)

In [5]:
# Checking the size of the tuning set and its composition by types of the selected tasks

print(len(evaluator_tuning_tasks))
evaluator_tuning_tasks.get_labels_counts()

50


{<TaskLabels.MERGE_VISUALIZE: 'Merge, Visualize'>: 8,
 <TaskLabels.TASK_SET_01: 'Task Set 01'>: 8,
 <TaskLabels.SPATIAL_OPERATIONS: 'Spatial operations'>: 16,
 <TaskLabels.TASK_SET_03: 'Task Set 03'>: 16,
 <TaskLabels.VAGUE: 'Vague'>: 4,
 <TaskLabels.HEATMAPS_CONTOUR_LINES: 'Heatmaps, Contour Lines'>: 14,
 <TaskLabels.TASK_SET_04: 'Task Set 04'>: 14,
 <TaskLabels.PROCESS_MERGE_VISUALIZE: 'Process, Merge, Visualize'>: 12,
 <TaskLabels.TASK_SET_02: 'Task Set 02'>: 12,
 <TaskLabels.HARD: 'Hard'>: 1}

In [ ]:
# saving the tuning set for evaluation if needed

evaluator_tuning_tasks.save_to_file(tuning_tasks_filename, DATA_FOLDER)

### Attention!

In the evaluator_tuning_set.json file in the repository, the LLM solutions are already generated. 

If a new file generated using the above part of the notebook:
1. Proceed with generating solutions by an LLM of choice and, 
2. Score it manually, by comparing the reference and candidate solutions using GUI by running tasks_editor or direclty in the json file, by inputting match score and match resoning under the keys "match_reasoning_Human" (optional) and "match_score_Human" (required) in the new file.

### Evaluate solutions in the tuning set

The LLM scores will be saved directly in the tuning tasks file. 

After scoring the set with an LLM, proceed to the next part of the notebook to calculate how close the LLM's evaluations are to the human scores.

Repeat for any LLM you plan to use for evaluations.

In [10]:
# Select the model to generate the solutions

model = MODEL_GPT
# Default temperature for evaluation of tasks is 0, to change temperature use line below. For OpenAI's o3-mini, use temperature = None

# temperature = 


In [11]:
model

'gpt-4.1-2025-04-14'

In [12]:
# Run evaluation of the whole set by the selected model
# The LLM match scores will be saved directly in the tuning tasks file. 
# Proceed to the next part of the notebook and see how close the LLM's solutions 

score_solutions_set(tuning_tasks_filename, DATA_FOLDER, model, skip_scored=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Task ID: TASK_250309_135125_530802
Task text: Map agricultural GDP contribution by region.


  2%|▏         | 1/50 [00:05<04:07,  5.05s/it]

Matching score: 1
input tokens: 15145, output tokens: 246
Task ID: TASK_250309_135125_315340
Task text: Map air quality index for major cities in South Asia?


  4%|▍         | 2/50 [00:07<02:45,  3.45s/it]

...

Matching score: 0
input tokens: 16058, output tokens: 588
Task ID: TASK_250309_135125_720013
Task text: Show electric consumption in G7 and BRICS nations


 98%|█████████▊| 49/50 [06:44<00:17, 17.21s/it]

Matching score: 0
input tokens: 15380, output tokens: 509
Task ID: TASK_250309_135125_908870
Task text: Visualize total population distribution by country.


100%|██████████| 50/50 [06:50<00:00,  8.20s/it]

Matching score: 2
input tokens: 15124, output tokens: 235
Total input tokens: 762313, total output tokens: 18372


### Evaluate comparisons

1. Count how many human comparisons and LLM's comparisons match. 

In [13]:
# Generate the matrix of LLM scores vs Human scores, percent of tasks for which the scores are the same and CI using Ward formula                                                                                                                                                                                     
generate_eval_stats_evaluator(tuning_tasks_filename, DATA_FOLDER)

         LLM_0  LLM_1  LLM_2
Human_0     23      1      0
Human_1      0      5      0
Human_2      1      1     19
0.94
(0.8378290831116182, 0.979385029651026)


In [ ]:
# Select tasks with particular combination of Human and LLM score
tasks_selected = [task for task in evaluator_tuning_tasks if task.match_score_LLM is not None and (task.match_score_Human.value, task.match_score_LLM.value)==(2, 0)]
tasks_selected

[|                                  | Task details                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
 |:---------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
# Select one task from the tuningv set to see the task details, reference and candidate solution and scores

check = [task for task in evaluator_tuning_tasks if task.task_ID == 'TASK_250309_135125_530802']
check

[|                                  | Task details                                                                                                                                                                                                                                                                                                     |
 |:---------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
 | task_ID                          | TASK_250309_135125_530802                                                                                                                                                                                                                                                       

### When ready, the evaluator code is used in the "Benchmarking" notebook.